# Exploratory Data Analysis

In [20]:
import pandas as pd
import json

# Load the JSON
with open('Dataset/Dogecoin 10 years.json', 'r') as file:
    data = json.load(file)

# Extract the 'prices' list (it's inside the first record)
# The JSON structure is [{'prices': [...], 'market_caps': [...], 'total_volumes': [...]}, ...]
# Each list contains sublists with [timestamp, value]

prices = data['prices']  # list of [timestamp, price]
market_caps = data['market_caps'] # list of [timestamp, market_cap]
volumes = data['total_volumes'] # list of [timestamp, volume]

# Convert to DataFrame
df_prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
df_caps = pd.DataFrame(market_caps, columns=['timestamp', 'market_cap'])
df_volumes = pd.DataFrame(volumes, columns=['timestamp', 'volume'])

# Merge on timestamp
df = df_prices.merge(df_caps, on='timestamp').merge(df_volumes, on='timestamp')

# Convert timestamp to datetime
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

# Rearrange columns
df = df[['date', 'price', 'market_cap', 'volume']]

# Preview
print(df.head())


        date     price  market_cap  volume
0 2013-12-15  0.000559   3488670.0     0.0
1 2013-12-17  0.000218   1619159.0     0.0
2 2013-12-18  0.000268   2191987.0     0.0
3 2013-12-19  0.000475   4299422.0     0.0
4 2013-12-20  0.000989   9866232.0     0.0
